# Capstone Project (Week 3)

## Segmenting and Clustering Neightbourhoods in Toronto

## Three Parts in one Notebook

# Part 1: Get data

We use the BeautifulSoup pacakage to scrape the table from the following webpage __[Toronto Data](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M)__
 
 The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood


In [173]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
df = pd.DataFrame(df[0])

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [174]:
df1 = df[df['Borough']!='Not assigned']

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [175]:
df1.reset_index
df1.index = range(len(df1.index))

for m in range(df1.shape[0]):
               if df1.loc[m]['Neighbourhood'] == 'Not assigned':
                   df1.loc[m]['Neighbourhood'] = df1.loc[m]['Borough']


More than one neighborhood can exist in one postal code area. These rows with same postal code will be combined into one row with the neighborhoods separated with a comma.

In [176]:
df2 = df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x)).reset_index()
df2 = df2.sort_values(['Postcode'])
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Check the shape of the data framework.

In [177]:
df2.shape

(103, 3)

# Part 2: find geophysical coordinates

Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [178]:
import geocoder
def get_latlng(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return lat_lng_coords
    
#get_latlng('M4G')
get_latlng('M1B')

[43.811525000000074, -79.19551746399998]

In [179]:
latAll = []
lngAll = []
for m in range(df2.shape[0]):
    postcode = df2.loc[m][0]
    latlng = get_latlng(postcode)
    lat = latlng[0]
    lng = latlng[1]
    latAll.append(lat)
    lngAll.append(lng)
                   

In [180]:
df2['Latitude'] =  latAll
df2['Longitude'] = lngAll

In [181]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


## Part 3: Clustering

We will now explore and cluster the neighborhoods in Toronto. We will work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data.

In [182]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library

address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [183]:
#ny_data = df2[df2['Borough'] == 'North York'].reset_index(drop=True)
ny_data = df2[df2['Borough'].str.contains("Toronto") ].reset_index(drop=True)
#address1 = 'North York,Toronto'
address1 = 'Toronto'

geolocator1 = Nominatim()
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
print('The geograpical coordinate of North York is {}, {}.'.format(latitude1, longitude1))



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


The geograpical coordinate of North York is 43.653963, -79.387207.


In [184]:
ny_data

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676531,-79.295425
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314667
3,M4M,East Toronto,Studio District,43.660629,-79.334855
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133
5,M4P,Central Toronto,Davisville North,43.712755,-79.388514
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960
7,M4S,Central Toronto,Davisville,43.703395,-79.385964
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.690685,-79.382946
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686074,-79.402265


In [185]:
map_ny = folium.Map(location=[latitude1, longitude1], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ny_data['Latitude'], ny_data['Longitude'], ny_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

#### Define Foursquare Credentials and Version

In [186]:
CLIENT_ID = 'C0XRAK52PJOJDZ002XEDSDULREZ50A2ZT0LORSV2EM1KY42F' # your Foursquare ID
CLIENT_SECRET = 'I2GPEGN0DHU5T0L3EYI4RCFPEHZZM1HJKP34BL5OF5R1JQ4X' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C0XRAK52PJOJDZ002XEDSDULREZ50A2ZT0LORSV2EM1KY42F
CLIENT_SECRET:I2GPEGN0DHU5T0L3EYI4RCFPEHZZM1HJKP34BL5OF5R1JQ4X


In [187]:
neighborhood_latitude = ny_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = ny_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = ny_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1, VERSION, radius, LIMIT)

Latitude and longitude values of The Beaches are 43.67653121600006, -79.29542499999997.


In [188]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [189]:
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
 
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Japango,Sushi Restaurant,43.655268,-79.385165
2,Sansotei Ramen 三草亭,Ramen Restaurant,43.655157,-79.386501
3,Cafe Plenty,Café,43.654571,-79.389450
4,Rolltation,Japanese Restaurant,43.654918,-79.387424
5,Manpuku まんぷく,Japanese Restaurant,43.653612,-79.390613
6,Tsujiri,Tea Room,43.655374,-79.385354
7,Textile Museum of Canada,Art Museum,43.654396,-79.386500
8,Poke Guys,Poke Place,43.654895,-79.385052
9,Karine's,Breakfast Spot,43.653699,-79.390743


In [190]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [191]:
ny_venues = getNearbyVenues(names=ny_data['Neighbourhood'],
                                   latitudes=ny_data['Latitude'],
                                   longitudes=ny_data['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

In [192]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighbourhood'] = ny_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_grouped = ny_onehot.groupby('Neighbourhood').mean().reset_index()

## Getting top 10 venues per neightbourhood

In [193]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [194]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = ny_grouped['Neighbourhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

### Run K-Means to get 5 clusters

In [195]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
    
neighbourhood_set = set(ny_grouped['Neighbourhood'].tolist())
idx_set = []
for m in range(24):
    if not ny_data['Neighbourhood'][m] in neighbourhood_set:
        idx_set.append(m)
        
ny_data1 = ny_data.drop(ny_data.index[idx_set])

# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [205]:
ny_merged = ny_data1

# add clustering labels
ny_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

ny_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,0,Health Food Store,Pub,Other Great Outdoors,Trail,Neighborhood,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105,0,Bus Line,Park,Discount Store,Grocery Store,Yoga Studio,Elementary School,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314667,0,Park,Italian Restaurant,Pet Store,Pizza Place,Pub,Coffee Shop,Movie Theater,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.660629,-79.334855,0,Diner,Italian Restaurant,Sushi Restaurant,Coffee Shop,Pizza Place,Brewery,Bar,Sandwich Place,Arts & Crafts Store,Grocery Store
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,0,Bus Line,Swim School,Yoga Studio,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
5,M4P,Central Toronto,Davisville North,43.712755,-79.388514,0,Food & Drink Shop,Clothing Store,Hotel,Breakfast Spot,Gym,Park,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
6,M4R,Central Toronto,North Toronto West,43.714523,-79.406960,0,Playground,Gym Pool,Park,Garden,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
7,M4S,Central Toronto,Davisville,43.703395,-79.385964,0,Dessert Shop,Coffee Shop,Café,Sandwich Place,Pizza Place,Italian Restaurant,Indian Restaurant,Seafood Restaurant,Sushi Restaurant,Restaurant
8,M4T,Central Toronto,"Moore Park,Summerhill East",43.690685,-79.382946,0,Gym,Convenience Store,Yoga Studio,Electronics Store,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
9,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686074,-79.402265,0,Light Rail Station,Coffee Shop,Photography Studio,Supermarket,Liquor Store,Elementary School,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


### Visualize the clusters

In [206]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude1, longitude1], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighbourhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters